In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [5]:
# Simulating simple node features and labels for now
# Later you can load real brain cancer graph data here

# Node features (5 nodes, 3 features per node)
x = torch.tensor([
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 0],
    [0, 0, 1],
    [1, 0, 0]
], dtype=torch.float)

# Edges (source -> target) [2 rows: [from], [to]]
edge_index = torch.tensor([
    [0, 1, 2, 3],
    [1, 2, 3, 4]
], dtype=torch.long)

# Labels for nodes (e.g. 0 = healthy, 1 = cancerous)
y = torch.tensor([0, 1, 1, 0, 1], dtype=torch.long)

data = Data(x=x, edge_index=edge_index, y=y)
print(data)


Data(x=[5, 3], edge_index=[2, 4], y=[5])


In [6]:
from torch.nn import Module

class GCN(Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(3, 4)  # 3 input features → 4 hidden
        self.conv2 = GCNConv(4, 2)  # 4 hidden → 2 output classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [7]:
model = GCN().to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, data.y)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 0.6768
Epoch 10, Loss: 0.6417
Epoch 20, Loss: 0.6320
Epoch 30, Loss: 0.6179
Epoch 40, Loss: 0.6033
Epoch 50, Loss: 0.5871
Epoch 60, Loss: 0.5672
Epoch 70, Loss: 0.5439
Epoch 80, Loss: 0.5184
Epoch 90, Loss: 0.4910


In [8]:
model.eval()
_, pred = model(data).max(dim=1)

print("Predicted:", pred.tolist())
print("Actual:   ", data.y.tolist())


Predicted: [0, 1, 1, 1, 1]
Actual:    [0, 1, 1, 0, 1]
